Simple SQL BlackJack by Daniel Janik - May 28 2019 - SQLTECHBLOG.COM

This is a sample notebook to show you some of the cool things what Azure Data Studio Notebooks can do while having a little fun.

You should start by running each block until you get to see your cards. 

Next you'll want to either Stay or Hit to try and win.

To reset the game: Click "Clear Results" and then run the code blocks in order from top to bottom.

Setup the deck. You could easily add more than one deck for added fun.

In [9]:
set nocount on
drop table if exists #deck

declare @suit table (suit nchar(1))
insert into @suit (suit) values (N'♠'),(N'♣'),(N'♥'),(N'♦')

declare @cardface table (type nchar(1), value tinyint)
insert into @cardface (type, value) values (N'1', 1),(N'2',2),(N'3',3),(N'4',4),(N'5',5),(N'6',6),(N'7',7),(N'8',8),(N'9',9),(N'J',10),(N'Q',10),(N'K',10),(N'A',11)

CREATE TABLE #deck (id int identity(1,1), type nchar(1), value tinyint, suit nchar(1), dealt bit default(0), playercard bit default(0))
insert into #deck (type, value, suit) 
select type, value, suit from @cardface cross apply @suit

Now deal cards to player and house. Using the rand() function helps simulate the deck shuffle.

In [10]:
--First card to the dealer
update #deck set dealt = 'TRUE' where id = FLOOR(RAND()*(52)+1);

--Second Card to the player
declare @i int = -1
while (@i = -1)
begin
	set @i = FLOOR(RAND()*(52)+1);
	if (@i in (select id from #deck where dealt = 'TRUE'))
	begin
		set @i = -1
	end
end

update #deck set dealt = 'TRUE', playercard = 'TRUE' where id = @i

--Third Card to the dealer
SET @i = -1
while (@i = -1)
begin
	set @i = FLOOR(RAND()*(52)+1);
	if (@i in (select id from #deck where dealt = 'TRUE'))
	begin
		set @i = -1
	end
end

update #deck set dealt = 'TRUE' where id = @i

--Fourth Card to the player
SET @i = -1
while (@i = -1)
begin
	set @i = FLOOR(RAND()*(52)+1);
	if (@i in (select id from #deck where dealt = 'TRUE'))
	begin
		set @i = -1
	end
end

update #deck set dealt = 'TRUE', playercard = 'TRUE' where id = @i

Show dealer and player cards

In [11]:
select type, suit from #deck where playercard = 'TRUE'

select top 1 type, suit from #deck where playercard = 'FALSE' and dealt = 'TRUE'
UNION
select '??', '??'

if (select sum(value) from #deck where playercard = 'TRUE') = 21
BEGIN
    print 'BLACKJACK! Player Wins.'
END

You can run either of the next two code blocks. Hit or Stay. Stay will evaluate your hand vs the house.

In [12]:
--Hit
declare @i int = -1
while (@i = -1)
begin
	set @i = FLOOR(RAND()*(52)+1);
	if (@i in (select id from #deck where dealt = 'TRUE'))
	begin
		set @i = -1
	end
end

update #deck set dealt = 'TRUE', playercard = 'TRUE' where id = @i

--Show player and dealer hands
select type, suit from #deck where playercard = 'TRUE'

select top 1 type, suit from #deck where playercard = 'FALSE' and dealt = 'TRUE'
UNION
select '??', '??'

declare @playervalue tinyint
select @playervalue = sum(value) from #deck where playercard = 'TRUE'
if (@playervalue > 21)
BEGIN
    while (@playervalue > 21) and exists (select id from #deck where playercard = 'TRUE' and value = 11)
    BEGIN
        --May count an ace as 1 instead of 11
        update #deck set value = 1 where id in (select top 1 id from #deck where playercard = 'TRUE' and value = 11)

        select @playervalue = sum(value) from #deck where playercard = 'TRUE'
    END
END

if (@playervalue > 21)
BEGIN
    print 'Player BUST!'
END

In [13]:
--Stay
--Show player and dealer hands
select type, suit from #deck where playercard = 'TRUE'

--If dealer hand is less than player hand dealer must draw cards until equal, higher, or until 21
declare @i int, @housevalue tinyint = 0, @playervalue tinyint = 0
select @playervalue = sum(value) from #deck where playercard = 'TRUE'
select @housevalue = sum(value) from #deck where dealt = 'TRUE' and playercard = 'FALSE'
IF (@housevalue > 21)
BEGIN
    PRINT 'House busted. Player Wins!'
END
else 
if (@housevalue >= @playervalue) 
BEGIN
    PRINT CONCAT('House Wins. ', @housevalue, ' to ', @playervalue) 
END
ELSE
BEGIN
    while (@housevalue < @playervalue)
    begin
        SET @i = -1
        while (@i = -1)
        begin
            set @i = FLOOR(RAND()*(52)+1);
            if (@i in (select id from #deck where dealt = 'TRUE'))
            begin
                set @i = -1
            end
        end

        update #deck set dealt = 'TRUE' where id = @i

        select @housevalue = sum(value) from #deck where dealt = 'TRUE' and playercard = 'FALSE'

        while (@housevalue > 21) and exists (select id from #deck where dealt = 'TRUE' and playercard = 'FALSE' and value = 11)
        BEGIN
            --May count an ace as 1 instead of 11
            update #deck set value = 1 where id in (select top 1 id from #deck where dealt = 'TRUE' and playercard = 'FALSE' and value = 11)

            select @housevalue = sum(value) from #deck where playercard = 'TRUE'
        END

    end
END

IF (@housevalue > 21)
BEGIN
    PRINT 'House busted. Player Wins!'
END
ELSE
if (@housevalue >= @playervalue) 
BEGIN
    PRINT CONCAT('House Wins. ', @housevalue, ' to ', @playervalue) 
END

select type, suit from #deck where playercard = 'FALSE' and dealt = 'TRUE'

Thanks for playing! If you're seeking a challenge and want to add to the game, why not add bets with a running player total? You may also consider a shoe to hold multiple decks or hand splitting... Enjoy!